## Import các thư viện cần thiết

In [ ]:
%autoawait True
%reload_ext autoreload
%autoreload 2
%autosave 120

import os
import glob
import random
import sys
sys.path.append("..")
from preprocess.word_segmentor import segment_sentences_into_words
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from gensim.models import Word2Vec

## Kiểm tra đường dẫn đến dataset

In [ ]:
data_path = "../data/raw/viwik18/dataset"

In [ ]:
!ls ../data/raw/viwik18/dataset

In [ ]:
filename_list = os.listdir(data_path)
filename_list.sort()
filename_list

In [ ]:
# Concat the filename with the data file path to get the full path of each file
filepath_list = [os.path.join(data_path, filename) for filename in filename_list]
filepath_list

In [ ]:
# Read the first file with encoding UTF-8.
# Replace error bytes with a placeholder character.
with open(filepath_list[0], "r", encoding="utf-8", errors="replace") as f:
    text = f.read()
text[-1000:]

In [ ]:
with open(filepath_list[1], "r", encoding="utf-8", errors="replace") as f:
    text = f.read()
text[:500]

### Vấn đề
Khi tải dữ liệu Wikipedia dạng _shard_ (chia thành nhiều file nhỏ), một ký tự UTF-8 có thể bị “cắt ngang” ở cuối file đầu tiên và tiếp tục ở đầu file tiếp theo. UTF-8 dùng từ 1–4 byte cho mỗi ký tự, nên nếu chỉ đọc từng file riêng lẻ, Python sẽ gặp `UnicodeDecodeError` vì nó thấy một byte mở đầu mà thiếu byte tiếp theo.

Cụ thể với `viwik18_aa`: byte cuối không đủ thông tin để tạo thành ký tự hợp lệ, dẫn đến lỗi decode.

### Giải pháp
Thay vì decode từng file một, ta nên nối tất cả các file ở mức byte (rb mode) rồi mới decode một lần. Như vậy các byte ký tự bị chia nhỏ ở ranh giới file sẽ được ghép lại đầy đủ, không còn gây lỗi.

In [ ]:
# Create a pattern to the data files
file_pattern = os.path.join(data_path, "viwik18_*")

# Search for all data files in the data directory
files = sorted(glob.glob(file_pattern))

In [ ]:
files

In [ ]:
all_bytes = b"".join(open(f, "rb").read() for f in files)
text = all_bytes.decode("utf-8")
split_text = text.split("  ")

In [ ]:
split_text[:100]

In [ ]:
# For each split, perform a .strip() on them
cleaned_splits = [s.strip() for s in split_text if s.strip()]

In [ ]:
# Set random seed
random.seed(42)

# Randomize 100 splits
randomized_splits = random.sample(cleaned_splits, 100)
randomized_splits[:10]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

In [ ]:
import time

start_time = time.time()
sentences = segment_sentences_into_words(cleaned_splits, nlp)
end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")

### Chuẩn bị các sentences để train model Word2Vec

In [ ]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
model.save("word2vec.model")